<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;"><img src="https://pbs.twimg.com/profile_images/1187259618/unidata_logo_rgb_sm_400x400.png" alt="Unidata Logo" style="height: 98px;"></div>

<h1>Making an RGB Satellite Composite</h1>
<h3>Unidata Python Workshop</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

<div style="float:right; width:250 px"><img src="https://unidata.github.io/MetPy/_images/sphx_glr_GINI_Water_Vapor_001.png" alt="Example Satellite Image" style="height: 500px;"></div>


## Overview:

* **Teaching:** 10 minutes
* **Exercises:** 20 minutes

### Questions
1. How can array manipulation be used with imagery data?
2. How can multiple channels of data be combined into an RGB image?
3. How can complex functionality be broken up into smaller reuseable and encapsulated functions?

### Objectives
1. Download satellite data with Siphon
2. Parse out netCDF file
3. Reshape and resample data channels to identical sizes
4. Create an RGB composite

Color images consist of red, green, and blue components. We can utilize the red and blue channels of the GOES-16 satellite, along with the “veggie” band to create a pseudo color image of the Earth. GOES-16 does not have a sensor that strictly covers the green visible range. Currently lookup tables are being created to simulate a true green sensor on the platform. Until those are completed, we will use the band as-is. It just results in an image that is too green.

We’ll need to accomplish a few tasks - making an outline like this is a good way to start thinking of how your program can be broken up into functions that encapsulate different functionality.

* Get three channels of satellite data
* Resample the red channel to match the resolution of the other channels
* Combine the RGB channels into a single multi-dimensional array
* Create the image


In [1]:
from datetime import datetime
from scipy import interpolate
from siphon.catalog import TDSCatalog
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from matplotlib import patheffects
import cartopy.feature as cfeat
import cartopy.crs as ccrs
import numpy as np

%matplotlib inline

In [ ]:
def open_dataset(date, channel, idx):
    """
    Open and return a netCDF Dataset object for a given date, channel, and image index
    of GOES-16 data from THREDDS test server.
    """
    cat = TDSCatalog('http://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GOES16/'
             '{}/CONUS/Channel{:02d}/catalog.xml'.format(date, channel))
    dataset_name = list(cat.datasets)[idx]
    dataset = cat.datasets[dataset_name]                       
    ds = Dataset(dataset.access_urls['OPENDAP'])
    return ds

def interpolate_red_channel(red_ds, resampled_ds):
    """
    Interpolate the red data channel to the same grid as another channel.
    """
    x_new = resampled_ds.variables['x'][:]
    y_new = resampled_ds.variables['y'][::-1]

    f = interpolate.interp2d(red_ds.variables['x'][:], red_ds.variables['y'][::-1],
                             red_ds.variables['Sectorized_CMI'][::-1,], fill_value=0)
    red_interpolated = f(x_new, y_new[::-1]) 
    return x_new, y_new, red_interpolated
    
def make_RGB_data(date, idx):
    """
    Make an RGB image array, returning the time, coordinates, projection, and data.
    """
    red_channel = 2
    green_channel = 3
    blue_channel = 1
                         
    red_ds = open_dataset(date, red_channel, idx)
    blue_ds = open_dataset(date, blue_channel, idx)
    green_ds = open_dataset(date, green_channel, idx)
                         
    green_data = green_ds.variables['Sectorized_CMI'][:]
    blue_data = blue_ds.variables['Sectorized_CMI'][:]

    x, y, red_data = interpolate_red_channel(red_ds, blue_ds)

    rgb_data = np.dstack([red_data[::-1,:], green_data, blue_data])

    x = green_ds.variables['x'][:]
    y = green_ds.variables['y'][:]  
    proj_var = green_ds.variables[green_ds.variables['Sectorized_CMI'].grid_mapping]

    time = datetime.strptime(green_ds.start_date_time, '%Y%j%H%M%S')
    
    return time, x, y, proj_var, rgb_data

In [ ]:
# Get the most recent image and make the RGB data array.
date = datetime.strftime(datetime.utcnow(), '%Y%m%d')
timestamp, x, y, proj_var, rgb_data = make_RGB_data(date, -1)

In [ ]:
# Create a Globe specifying a spherical earth with the correct radius
globe = ccrs.Globe(ellipse='sphere', semimajor_axis=proj_var.semi_major,
                   semiminor_axis=proj_var.semi_minor)

proj = ccrs.LambertConformal(central_longitude=proj_var.longitude_of_central_meridian,
                             central_latitude=proj_var.latitude_of_projection_origin,
                             standard_parallels=[proj_var.standard_parallel],
                             globe=globe)


# Set up a feature for the state/province lines. Tell cartopy not to fill in the polygons
state_boundaries = cfeat.NaturalEarthFeature(category='cultural',
                                             name='admin_1_states_provinces_lakes',
                                             scale='50m', facecolor='none')

# Same as before, except we call imshow with our colormap and norm.
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(1, 1, 1, projection=proj)

im = ax.imshow(rgb_data, extent=(x.min(), x.max(), y.min(), y.max()), origin='upper')
ax.coastlines(resolution='50m', color='black')
ax.add_feature(state_boundaries, linestyle=':', edgecolor='black')
ax.add_feature(cfeat.BORDERS, linewidth='2', edgecolor='black')

# Add text (aligned to the right); save the returned object so we can manipulate it.
text_time = ax.text(0.99, 0.01, timestamp.strftime('%d %B %Y %H%MZ'),
               horizontalalignment='right', transform=ax.transAxes,
               color='white', fontsize='x-large', weight='bold')

text_channel = ax.text(0.5, 0.97, 'Experimental GOES-16 RGB Composite',
               horizontalalignment='center', transform=ax.transAxes,
               color='white', fontsize='large', weight='bold')

# Make the text stand out even better using matplotlib's path effects
text_time.set_path_effects([patheffects.Stroke(linewidth=2, foreground='black'),
                       patheffects.Normal()])
text_channel.set_path_effects([patheffects.Stroke(linewidth=2, foreground='black'),
                       patheffects.Normal()])
ax.set_extent([240., 290., 20., 49.])

## Image Filtering

In [ ]:
from skimage.morphology import reconstruction

seed = np.copy(rgb_data)
seed[1:-1, 1:-1] = rgb_data.max()
mask = rgb_data

filled = reconstruction(seed, mask, method='erosion')

In [ ]:
# Create a Globe specifying a spherical earth with the correct radius
globe = ccrs.Globe(ellipse='sphere', semimajor_axis=proj_var.semi_major,
                   semiminor_axis=proj_var.semi_minor)

proj = ccrs.LambertConformal(central_longitude=proj_var.longitude_of_central_meridian,
                             central_latitude=proj_var.latitude_of_projection_origin,
                             standard_parallels=[proj_var.standard_parallel],
                             globe=globe)


# Set up a feature for the state/province lines. Tell cartopy not to fill in the polygons
state_boundaries = cfeat.NaturalEarthFeature(category='cultural',
                                             name='admin_1_states_provinces_lakes',
                                             scale='50m', facecolor='none')

# Same as before, except we call imshow with our colormap and norm.
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(1, 1, 1, projection=proj)

im = ax.imshow(filled, extent=(x.min(), x.max(), y.min(), y.max()), origin='upper')
ax.coastlines(resolution='50m', color='black')
ax.add_feature(state_boundaries, linestyle=':', edgecolor='black')
ax.add_feature(cfeat.BORDERS, linewidth='2', edgecolor='black')

# Add text (aligned to the right); save the returned object so we can manipulate it.
text_time = ax.text(0.99, 0.01, timestamp.strftime('%d %B %Y %H%MZ'),
               horizontalalignment='right', transform=ax.transAxes,
               color='white', fontsize='x-large', weight='bold')

text_channel = ax.text(0.5, 0.97, 'Experimental GOES-16 RGB Composite',
               horizontalalignment='center', transform=ax.transAxes,
               color='white', fontsize='large', weight='bold')

# Make the text stand out even better using matplotlib's path effects
text_time.set_path_effects([patheffects.Stroke(linewidth=2, foreground='black'),
                       patheffects.Normal()])
text_channel.set_path_effects([patheffects.Stroke(linewidth=2, foreground='black'),
                       patheffects.Normal()])
ax.set_extent([240., 290., 20., 49.])